# MTH5 to StationXML and miniseed

In [1]:
from pathlib import Path
from mth5.mth5 import MTH5
from mt_metadata.timeseries.stationxml import XMLInventoryMTExperiment

In [2]:
network_code = "8P"

In [3]:
h5_path = Path(r"c:\Users\jpeacock\OneDrive - DOI\MTData\CL2021\archive\cl008.h5")

In [4]:
with MTH5() as m:
    m.open_mth5(h5_path)
    experiment = m.to_experiment()
    stream_list = []
    for row in m.run_summary.itertuples():
        if row.has_data:
            run_ts = m.from_reference(row.run_hdf5_reference).to_runts()
            stream_list.append(run_ts.to_obspy_stream(network_code=network_code))

2025-03-06T15:46:42.228695-0800 | INFO | mth5.timeseries.run_ts | _align_channels | Channels do not have a common end, using latest: 2021-04-30T18:59:43.996094000
2025-03-06T15:46:48.474863-0800 | INFO | mth5.timeseries.run_ts | _align_channels | Channels do not have a common end, using latest: 2021-04-30T19:09:43.998779296
2025-03-06T15:46:50.998548-0800 | WARNING | mth5.timeseries.run_ts | validate_metadata | end time of dataset 2021-04-30T19:09:43.998535156+00:00 does not match metadata end 2021-04-30T19:09:43.998779296+00:00 updating metatdata value to 2021-04-30T19:09:43.998535156+00:00
2025-03-06T15:46:58.484151-0800 | INFO | mth5.timeseries.run_ts | _align_channels | Channels do not have a common end, using latest: 2021-05-01T00:59:43.996094000
2025-03-06T15:47:08.835768-0800 | INFO | mth5.timeseries.run_ts | _align_channels | Channels do not have a common end, using latest: 2021-05-01T01:09:43.997803000
2025-03-06T15:47:18.810549-0800 | INFO | mth5.timeseries.run_ts | _align_ch

In [5]:
stream_list

[5 Trace(s) in Stream:
8P.cl008..LQN | 2021-04-30T16:19:59.000000Z - 2021-05-01T15:13:34.000000Z | 1.0 Hz, 82416 samples
8P.cl008..LQE | 2021-04-30T16:19:59.000000Z - 2021-05-01T15:13:34.000000Z | 1.0 Hz, 82416 samples
8P.cl008..LFN | 2021-04-30T16:19:59.000000Z - 2021-05-01T15:13:34.000000Z | 1.0 Hz, 82416 samples
8P.cl008..LFE | 2021-04-30T16:19:59.000000Z - 2021-05-01T15:13:34.000000Z | 1.0 Hz, 82416 samples
8P.cl008..LFZ | 2021-04-30T16:19:59.000000Z - 2021-05-01T15:13:34.000000Z | 1.0 Hz, 82416 samples,
 5 Trace(s) in Stream:
8P.cl008..CQN | 2021-04-30T16:19:59.000000Z - 2021-04-30T18:59:43.996094Z | 256.0 Hz, 2453760 samples
8P.cl008..CQE | 2021-04-30T16:19:59.000000Z - 2021-04-30T18:59:43.996094Z | 256.0 Hz, 2453760 samples
8P.cl008..CFN | 2021-04-30T16:19:59.000000Z - 2021-04-30T18:59:43.996094Z | 256.0 Hz, 2453760 samples
8P.cl008..CFE | 2021-04-30T16:19:59.000000Z - 2021-04-30T18:59:43.996094Z | 256.0 Hz, 2453760 samples
8P.cl008..CFZ | 2021-04-30T16:19:59.000000Z - 2021-04-3

In [ ]:
m.h5_is_read

array([26370590., 26385844., 26417756., ..., 27133124., 27163860.,
             nan])

In [8]:
s1.write(h5_path.parent.joinpath("s1_test.mseed"), format="MSEED", reclen=256)

c:\Users\jpeacock\AppData\Local\miniforge3\envs\py311\Lib\site-packages\obspy\io\mseed\core.py:824: UserWarning: File will be written with more than one different encodings.
This might have a negative influence on the compatibility with other programs.
  warnings.warn(msg % 'encodings')


In [11]:
experiment.surveys[0].fdsn.network = network_code

translator = XMLInventoryMTExperiment()
stationxml = translator.mt_to_xml(
    experiment, stationxml_fn=h5_path.parent.joinpath("test_cl008.xml")
)

In [8]:
import pandas as pd

df = pd.read_csv(r"c:\Users\jpeacock\OneDrive - DOI\MTData\CL2021\survey_summary.csv")

In [24]:
for h5_fn in h5_path.parent.glob("*h5"): 
    with MTH5() as m:
        m.open_mth5(h5_fn)
        for row in m.run_summary.itertuples():
            run_group = m.from_reference(row.run_hdf5_reference)
            run_group.metadata.data_logger.id = df.loc[df.station == h5_fn.stem]["instrument_id"].array[0]
            run_group.metadata.data_logger.manufacturer = "Zonge International"
            run_group.write_metadata()
        # ch_summary = m.channel_summary.to_dataframe()
        # mags = ch_summary[ch_summary.measurement_type == "magnetic"]
        # for row in mags.itertuples():
        #     ch = m.from_reference(row.hdf5_reference)
        #     for key in ["id", "manufacturer", "model", "type"]:
        #         attr = f"sensor.{key}"
        #         value = ch.metadata.get_attr_from_name(attr)
        #         if isinstance(value, (list, np.ndarray)):
        #             ch.metadata.set_attr_from_name(
        #                 attr, ch.metadata.get_attr_from_name(attr)[0]
        #             )
        #     ch.write_metadata()

2025-03-06T14:59:05.702589-0800 | INFO | mth5.mth5 | close_mth5 | Flushing and closing c:\Users\jpeacock\OneDrive - DOI\MTData\CL2021\archive\cl008.h5
2025-03-06T14:59:09.948523-0800 | INFO | mth5.mth5 | close_mth5 | Flushing and closing c:\Users\jpeacock\OneDrive - DOI\MTData\CL2021\archive\cl016.h5
2025-03-06T14:59:13.904538-0800 | INFO | mth5.mth5 | close_mth5 | Flushing and closing c:\Users\jpeacock\OneDrive - DOI\MTData\CL2021\archive\cl017.h5
2025-03-06T14:59:16.925630-0800 | INFO | mth5.mth5 | close_mth5 | Flushing and closing c:\Users\jpeacock\OneDrive - DOI\MTData\CL2021\archive\cl024.h5
2025-03-06T14:59:20.455742-0800 | INFO | mth5.mth5 | close_mth5 | Flushing and closing c:\Users\jpeacock\OneDrive - DOI\MTData\CL2021\archive\cl028.h5
2025-03-06T14:59:23.396660-0800 | INFO | mth5.mth5 | close_mth5 | Flushing and closing c:\Users\jpeacock\OneDrive - DOI\MTData\CL2021\archive\cl029.h5
2025-03-06T14:59:26.479024-0800 | INFO | mth5.mth5 | close_mth5 | Flushing and closing c:\User

In [23]:
df.loc[df.station == "cl306"]["instrument_id"].array[0]


'ZEN_026'